<div style="text-align: center; font-size: 36px; font-weight: bold; color: #4a86e8;">
Pre-processing flow
</div>

In [1]:
import numpy as np
import os
import nibabel as nib
from nilearn import image
from nilearn.image import resample_to_img
#import fsl.wrappers.bet as bet

In [2]:
#Setup FSL environment (for brain extraction)

# Ensure FSLDIR is set
os.environ['FSLDIR'] = '/Users/imeag/fsl'
os.environ['PATH'] = os.environ['FSLDIR'] + '/bin:' + os.environ['PATH']

# Set FSLOUTPUTTYPE environment variable
#os.environ['FSLOUTPUTTYPE'] = 'NIFTI_GZ'
os.environ['FSLOUTPUTTYPE'] = 'NIFTI'


<div style="text-align: center; font-size: 36px; font-weight: bold; color: #4a86e8;">
Function load_nii
</div>

<br>

<div style="text-align: justify; font-size: 18px; line-height: 1.6;">
The <span style="color: #4a86e8; font-weight: bold;">cargar_nii</span> function is designed to load a NIfTI (.nii) file and return the image as a NumPy array. This versatile function performs the following operations:

- Takes a file path (archivo) as an argument
- Uses the nibabel library to load the NIfTI image
- Extracts the image data using the get_fdata() method
- Converts the image into a NumPy array
- Returns this array for further processing
</div>

<br>

In [2]:
def load_nii(file_path):
    """
    Loads a .nii (NIfTI) file and returns the image as a NumPy array.
   
    :param file_path: Path to the .nii file
    :return: Loaded image as a NumPy array
    """
    nifti_image = nib.load(file_path)
    image_array = nifti_image.get_fdata()
    return image_array


<div style="text-align: center; font-size: 36px; font-weight: bold; color: #4a86e8;">
Function reorient_image
</div>

<br>

<div style="text-align: justify; font-size: 18px; line-height: 1.6;">
The <span style="color: #4a86e8; font-weight: bold;">reorient_image</span> function is designed to reorient an image to match the orientation of a reference image and save the result. This function performs the following operations:

- Takes three file paths as arguments: reference image, image to reorient, and output path
- Verifies the existence of input files
- Loads both the reference and target images using nibabel
- Displays basic information about the loaded images
- Reorients the target image to match the reference image
- Saves the reoriented image to the specified output path
</div>

<br>

In [4]:
def reorient_image(reference_image_path, image_to_reorient_path, output_path):
    """
    Reorienta una imagen a la orientación de una imagen de referencia y guarda el resultado.

    Parameters:
    - reference_image_path (str): Ruta del archivo .nii de referencia.
    - image_to_reorient_path (str): Ruta del archivo .nii que se quiere reorientar.
    - output_path (str): Ruta donde se guardará la imagen reorientada.
    """
    # Verificar si los archivos existen
    if not (os.path.exists(reference_image_path) and os.path.exists(image_to_reorient_path)):
        raise FileNotFoundError("Uno o ambos archivos no se encontraron.")

    try:
        # Cargar las imágenes
        imagen1 = nib.load(image_to_reorient_path)  # Imagen a reorientar
        imagen2 = nib.load(reference_image_path)  # Imagen de referencia
        
        # Comprobar los tipos de datos
        print("Tipo de imagen a reorientar:", type(imagen1))  # Debería mostrar <class 'nibabel.nifti1.Nifti1Image'>
        print("Tipo de imagen de referencia:", type(imagen2))  # Debería mostrar <class 'nibabel.nifti1.Nifti1Image'>
        
        # Mostrar información básica de las imágenes
        print("Imagen a reorientar:")
        print("Tipo de datos:", imagen1.get_data_dtype())
        print("Forma:", imagen1.shape)
        
        print("\nImagen de referencia:")
        print("Tipo de datos:", imagen2.get_data_dtype())
        print("Forma:", imagen2.shape)

        # Reorientar la imagen
        reoriented_img = resample_to_img(imagen1, imagen2, interpolation='nearest')

        # Guardar la imagen reorientada
        reoriented_img.to_filename(output_path)

        print(f"Reorientación completa y imagen guardada como '{output_path}'.")

    except Exception as e:
        print(f"Error al cargar las imágenes o realizar la reorientación: {e}")

<div style="text-align: center; font-size: 36px; font-weight: bold; color: #4a86e8;">
Function normalize_image
</div>

<br>

<div style="text-align: justify; font-size: 18px; line-height: 1.6;">
The <span style="color: #4a86e8; font-weight: bold;">normalize_image</span> function is designed to normalize the grayscale of an image and save the result. This function performs the following operations:

- Takes two file paths as arguments: input image path and output path
- Loads the image using nibabel
- Normalizes the grayscale of the image
- Creates a new image with the normalized data
- Saves the normalized image to the specified output path
</div>

<br>

<div style="text-align: center;">
<img src="assets/normalize_image.png" alt="Image normalization process" style="max-width: 100%; height: auto;">
</div>

<br>

In [5]:
def normalize_image(image_path, output_path):
    """
    Normalizes the grayscale of an image and saves the result.
    Parameters:
    - image_path (str): Path to the .nii file of the image.
    - output_path (str): Path where the normalized image will be saved.
    """
    # Load the image
    img = nib.load(image_path)
    data = img.get_fdata()
    
    # Normalize the grayscale
    min_val = np.min(data)
    max_val = np.max(data)
    normalized_data = (data - min_val) / (max_val - min_val)
    
    # Create a new image with the normalized data
    normalized_img = nib.Nifti1Image(normalized_data, img.affine, img.header)
   
    # Save the normalized image
    normalized_img.to_filename(output_path)
    print(f"Normalized image saved as '{output_path}'.")

---
<div style="text-align: center; font-size: 36px; font-weight: bold; color: #4a86e8;">
extract_brain Function
</div>

<br>

<div style="text-align: justify; font-size: 18px; line-height: 1.6;">
The <span style="color: #4a86e8; font-weight: bold;">extract_brain</span> function is designed to extract the brain from a NIfTI (.nii) image and save the result to a file. It performs the following operations:

- Takes an input file path (<span style="font-weight: bold;">input_path</span>) that points to the original NIfTI image.
- Takes an output file path (<span style="font-weight: bold;">output_path</span>) where the image with the extracted brain will be saved.
- Uses a function called <span style="font-weight: bold;">bet</span> to extract the brain with the fraction parameter set to f=0.32.
- Prints a message confirming that the brain extraction is complete and the file has been saved to the specified location.

</div>

<br>

<br>

---

In [ ]:
def extract_brain(input_path, output_path):
    """
    Extracts the brain from an image and saves the result.
    
    Parameters:
    - input_path (str): Path to the input .nii file of the image.
    - output_path (str): Path where the image with the extracted brain will be saved.
    """
    # Extract the brain
    result = bet(input_path, output_path, f=0.32)
    print(f"Brain extraction complete and result saved as '{output_path}'.")


<div style="text-align: center; font-size: 36px; font-weight: bold; color: #4a86e8;">
apply_gaussian_filter Function
</div>

<br>

<div style="text-align: justify; font-size: 18px; line-height: 1.6;">
The <span style="color: #4a86e8; font-weight: bold;">apply_gaussian_filter_nifti</span> function is designed to apply a Gaussian filter to T2-weighted MRI images in NIfTI format and save the resulting image. This function performs the following operations:

- Loads a NIfTI image from the specified input path.
- Applies a Gaussian filter to the image data using a specified standard deviation (<span style="font-weight: bold;">sigma</span>).
- Creates a new NIfTI image with the filtered data.
- Saves the filtered image to the specified output path.

</div>

<br>

In [ ]:
def apply_gaussian_filter(input_path, output_path, sigma):
    """
    Applies a Gaussian filter to a T2-weighted MRI image in NIfTI format and saves the result.
    
    Parameters:
    - input_path (str): Path to the input NIfTI file.
    - output_path (str): Path where the filtered NIfTI image will be saved.
    - sigma (float): Standard deviation for the Gaussian kernel.
    """
    # Load the NIfTI image
    nifti_image = nib.load(input_path)
    image_data = nifti_image.get_fdata()

    # Apply the Gaussian filter
    filtered_image = gaussian_filter(image_data, sigma=sigma)

    # Create a new NIfTI image with the filtered data
    filtered_nifti = nib.Nifti1Image(filtered_image, affine=nifti_image.affine)

    # Save the filtered image to the specified output path
    nib.save(filtered_nifti, output_path)

    print(f"Filtered image saved as '{output_path}'.")

